<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/Langchain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain cohere langchain-huggingface bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.4 MB/s eta 0:00:00


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `langt_token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentic

In [7]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('hf_token'))

In [8]:
import torch
import transformers
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline

In [10]:
model_name='mistralai/Mistral-7B-v0.1'

model_config = transformers.AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True

bnb_4bit_compute_dtype = "float16"  # 16floating point compute type
bnb_4bit_quant_type = "nf4"  # 4 bit quant type
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=512,
    output_scores=True
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


## Sequential Chain

In [11]:
template = """
I want you to act as a expert who can generate details on the given scientific process {topic}.
"""

prompt_template = PromptTemplate(
    input_variables=["topic"],
    template=template,
)

chain = LLMChain(llm=mistral_llm, prompt=prompt_template)

<ipython-input-11-c6056b9fac43>:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=mistral_llm, prompt=prompt_template)


In [12]:
description = "Photosynthesis"
prompt_template.format(topic=description)

'\nI want you to act as a expert who can generate details on the given scientific process Photosynthesis.\n'

In [13]:
print(chain.invoke(input={'topic':description}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(


{'topic': 'Photosynthesis', 'text': '\nI want you to act as a expert who can generate details on the given scientific process Photosynthesis.\n\nThe process of photosynthesis is a process that is carried out by plants and other organisms. It is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to conver

In [15]:
input = "oxidation"
prompt_template.format(topic=input)

'\nI want you to act as a expert who can generate details on the given scientific process oxidation.\n'

In [16]:
print(chain.invoke(input={'topic':input}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(


{'topic': 'oxidation', 'text': '\nI want you to act as a expert who can generate details on the given scientific process oxidation.\n\nThe process of oxidation is a chemical reaction that involves the loss of electrons by one substance and the gain of electrons by another substance. Oxidation can occur when a substance reacts with oxygen, or when it reacts with another substance that has been oxidized.\n\nOxidation is a chemical reaction that occurs when a substance reacts with oxygen. This can happen when a substance is exposed to air, or when it is exposed to another substance that has been oxidized. Oxidation can also occur when a substance reacts with another substance that has been oxidized.\n\nOxidation can be a chemical reaction that occurs when a substance reacts with oxygen. This can happen when a substance is exposed to air, or when it is exposed to another substance that has been oxidized. Oxidation can also occur when a substance reacts with another substance that has been 